# Tarea 6

Las familias de modelos codificadores, tales como BERT, ALBERT, DistilBERT, ELECTRA, RoBERTA, y MPnet, han emergido como pilares fundamentales en el campo de procesamiento de lenguaje natural. Estos modelos comparten la arquitectura de transformer y han sido preentrenados para aprender representaciones semánticas profundas del lenguaje.

Comencemos analizando BERT, un pionero en la atención bidireccional. BERT, en sus múltiples variantes, como BERT-base y BERT-large, destaca por su capacidad para capturar contextos ricos y relaciones sintácticas. Por otro lado, ALBERT introduce la innovación de la factorización de la matriz de atención, optimizando así la eficiencia computacional sin comprometer el rendimiento. La comparación entre BERT y ALBERT revela el equilibrio entre complejidad y eficiencia.

DistilBERT, por su parte, busca la simplicidad sin sacrificar la capacidad representativa. Al destilar el conocimiento de un modelo más grande como BERT, logra una reducción significativa en los parámetros, haciéndolo más liviano y ágil. Esta característica lo posiciona como una opción atractiva en tareas con recursos computacionales limitados.

ELECTRA, con su aproximación novedosa, implementa un enfoque de sustitución de palabras para el entrenamiento discriminatorio. Al destacar la eficacia de este método, ELECTRA demuestra ser competitivo en la obtención de representaciones de alta calidad.

RoBERTa, una mejora de BERT, ajusta la metodología de preentrenamiento al eliminar la tarea de predicción de la orientación de la siguiente palabra. Esto conduce a representaciones más coherentes y robustas, destacando su eficacia en diversos contextos lingüísticos.

Finalmente, MPnet se destaca por su enfoque en la modelación de patrones a nivel de múltiples posiciones. Su capacidad para capturar información contextual a lo largo de múltiples niveles jerárquicos proporciona una perspectiva única en la representación del lenguaje.

In [1]:
from collections import defaultdict, Counter
import json

from matplotlib import pyplot as plt
import numpy as np
import torch

In [2]:
import os
import random
import re
from transformers import AutoTokenizer
from datasets import Dataset, DatasetDict
import nltk

# Download the NLTK Punkt tokenizer if not already downloaded
nltk.download('punkt')

# Function to create integer label for authors
def encode_author(author, authors_list):
    return authors_list.index(author)

# Function to clean and preprocess text
def clean_text(text):
    # Remove unwanted characters, including newline
    text = text.replace("\n", " ")
    return text

# Function to tokenize text into sentences
def tokenize_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

# Define authors and paths
authors = os.listdir("book_datasets")
data = {'text': [], 'label': []}

# Iterate over authors and text files
for author in authors:
    author_path = os.path.join("book_datasets", author)
    author_files = [f for f in os.listdir(author_path) if f.endswith(".txt")]

    for file in author_files:
        file_path = os.path.join(author_path, file)
        
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
            
            # Clean and preprocess the text
            text = clean_text(text)
            
            # Tokenize text into sentences
            sentences = tokenize_into_sentences(text)
            
            # Create integer label for each sentence
            for sentence in sentences:
                label = encode_author(author, authors)
                data['text'].append(sentence)
                data['label'].append(label)

# Shuffle the dataset
random.shuffle(data['text'])
random.shuffle(data['label'])

# Split the dataset into training and validation sets
train_size = int(0.8 * len(data['text']))
train_data = {'text': data['text'][:train_size], 'label': data['label'][:train_size]}
val_data = {'text': data['text'][train_size:], 'label': data['label'][train_size:]}

# Create datasets
book_train_dataset = Dataset.from_dict(train_data)
book_val_dataset = Dataset.from_dict(val_data)

# Create a DatasetDict
book_dataset = DatasetDict({'train': book_train_dataset, 'val': book_val_dataset})

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
book_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 42889
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 10723
    })
})

In [4]:
book_dataset["train"]["text"][5050]

'Fiery?'

In [5]:
book_dataset["train"]["label"][5050]

0

In [6]:
np.unique(book_dataset["train"]["label"])

array([0, 1, 2])

In [7]:
# with 'train' and 'val' splits
train_labels = book_dataset['train']['label']
val_labels = book_dataset['val']['label']

# Compute label counts for the 'train' split
train_label_counts = dict(zip(*np.unique(train_labels, return_counts=True)))

# Compute label counts for the 'val' split
val_label_counts = dict(zip(*np.unique(val_labels, return_counts=True)))

print("Train Label Counts:")
print(train_label_counts)

print("\nVal Label Counts:")
print(val_label_counts)


Train Label Counts:
{0: 15325, 1: 12780, 2: 14784}

Val Label Counts:
{0: 3730, 1: 3258, 2: 3735}


In [8]:
# Tokenize dataset
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
tokenized_dataset = book_dataset.map(lambda example: tokenizer(example['text'], truncation=True), batched=True)

tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

Map: 100%|██████████| 10723/10723 [00:00<00:00, 34547.21 examples/s]


In [9]:
tokenized_dataset["train"][0:2]

{'labels': tensor([2, 2]),
 'input_ids': [tensor([  101,   143,  2346, 14697,  2249, 14697,  9919,   196,   121,   198,
            168, 20777,  4854,  1906, 23231,   168,  1108,  1308,  1112,   170,
           2767,   174,  1942, 11708,  1204,  1118,  4042,   144,  6140,  8904,
            117,  1105,  1110,  1136,  1529,  1107,  1142,   174,  1942, 11708,
           1204,   119,   102]),
  tensor([  101,   789,  1573,   117,  1128,   787,  1231,  1280,  1113,  1106,
           5738,  1944,  8914,  6944,   117,  1132,  1128,   136,   790, 28110,
          14159,  9029,   119,   102])],
 'attention_mask': [tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]}

In [10]:
# Specify GPU device for Trainer
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
device

'cuda'

In [12]:
from transformers import TrainerCallback, TrainingArguments, Trainer, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers.integrations import TensorBoardCallback

# Load pre-trained model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=3)
model.to(device)

# Training arguments
arguments = TrainingArguments(
    output_dir="sample_hf_trainer",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=20,
    evaluation_strategy="epoch",  # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224,
    logging_dir="sample_hf_trainer/logs",  # Directory for Tensorboard logs
    logging_steps=100,  # Log every 100 steps
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}

# Create trainer with loaded pre-trained model
trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
        self.train_loss = []
        self.val_loss = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")
            
            # Append training and validation loss to lists
            self.train_loss.append(logs.get("loss", np.inf))
            self.val_loss.append(logs.get("eval_loss", np.inf))

In [14]:
trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Add the callbacks to the trainer
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.0))
trainer.add_callback(LoggingCallback("sample_hf_trainer/log.jsonl"))
trainer.add_callback(TensorBoardCallback())

You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
ProgressCallback
EarlyStoppingCallback
LoggingCallback


In [15]:
trainer.train()

  0%|          | 100/71500 [00:12<1:48:28, 10.97it/s]

{'loss': 1.1008, 'learning_rate': 1.9972027972027975e-05, 'epoch': 0.03}


  0%|          | 201/71500 [00:22<2:07:03,  9.35it/s]

{'loss': 1.1025, 'learning_rate': 1.9944055944055948e-05, 'epoch': 0.06}


  0%|          | 300/71500 [00:33<1:45:55, 11.20it/s]

...

                                                       
 15%|█▌        | 10725/71500 [24:41<2:07:22,  7.95it/s]

{'eval_loss': 1.1313278675079346, 'eval_accuracy': 0.3488762473188473, 'eval_runtime': 48.1243, 'eval_samples_per_second': 222.819, 'eval_steps_per_second': 18.577, 'epoch': 3.0}


 15%|█▌        | 10725/71500 [24:43<2:20:04,  7.23it/s]

{'train_runtime': 1483.2063, 'train_samples_per_second': 578.328, 'train_steps_per_second': 48.206, 'train_loss': 1.0910380687580241, 'epoch': 3.0}


TrainOutput(global_step=10725, training_loss=1.0910380687580241, metrics={'train_runtime': 1483.2063, 'train_samples_per_second': 578.328, 'train_steps_per_second': 48.206, 'train_loss': 1.0910380687580241, 'epoch': 3.0})